In [20]:
import pandas as pd 

In [21]:
year = '2017'

In [22]:
df = pd.read_csv('points_against_'+year+'.csv')

In [23]:
df.loc[df.Opponent.str.contains("@",na=False),'Home_Away'] = 'Away'
df.loc[df.Home_Away.isnull(), 'Home_Away'] = 'Home'
df.Opponent = df.Opponent.str.replace('@', '')

In [24]:
df['Game_Dev'] = (df.Points_Against.astype(float) - df.Season_Ave.astype(float))/df.Season_Ave.astype(float)

In [25]:
df_opponent = df[['Opponent','Position','Year']]
df_opponent = df_opponent.drop_duplicates()
df_opponent = df_opponent[df_opponent.Opponent != 'Bye']
df_opponent.columns = ['Team','Position','Year']

In [26]:
df_opponent['ave'] = 0
df_opponent['std'] = 0
for index, row in df_opponent.iterrows():
    df_opponent.loc[(df_opponent['Team'] == row.Team) & (df_opponent.Position == row.Position) & (df_opponent.Year == row.Year),'ave'] = df[(df.Opponent == row.Team) & (df.Position == row.Position) & (df.Year == row.Year)].Game_Dev.mean()
    df_opponent.loc[(df_opponent['Team'] == row.Team) & (df_opponent.Position == row.Position) & (df_opponent.Year == row.Year),'std'] = df[(df.Opponent == row.Team) & (df.Position == row.Position) & (df.Year == row.Year)].Game_Dev.std()

In [31]:
def matchup(offense, defense):
    df_off = df[(df['Team'] == defense)][['Team','Position','Season_Ave','Season_STD']]
    df_off = df_off.drop_duplicates()
    df_off.columns = ['Defense','Position','Season_Ave','Season_STD']
    df_def = df_opponent[df_opponent['Team']==offense]
    df_def.columns = ['Offense','Position','Year','Ave','STD']
    df_matchup = pd.merge(left=df_off, right=df_def, left_on='Position', right_on='Position')
    df_matchup['16th_Percentile'] = df_matchup['Season_Ave'] - df_matchup['Season_STD']
    df_matchup['84th_Percentile'] = df_matchup['Season_Ave'] + df_matchup['Season_STD']
    df_matchup['Lower_STD_Pred'] = (df_matchup['Ave']+1-df_matchup['STD'])*df_matchup['Season_Ave']
    df_matchup['Mean_Pred'] = (df_matchup['Ave']+1)*df_matchup['Season_Ave']
    df_matchup['Upper_STD_Pred'] = (df_matchup['Ave']+1+df_matchup['STD'])*df_matchup['Season_Ave']
    df_matchup = df_matchup[['Offense','Defense','Position','16th_Percentile','84th_Percentile','Lower_STD_Pred','Mean_Pred','Upper_STD_Pred']]
    return df_matchup
    

In [32]:
matchup('GB', 'Cincinnati Bengals Defense')

,Offense,Defense,Position,16th_Percentile,84th_Percentile,Lower_STD_Pred,Mean_Pred,Upper_STD_Pred
0,GB,Cincinnati Bengals Defense,QB,4.590820,19.949180,13.130249,18.290932,23.451616
1,GB,Cincinnati Bengals Defense,RB,4.590820,19.949180,13.130249,18.290932,23.451616
2,GB,Cincinnati Bengals Defense,WR,12.125379,26.974621,17.336019,17.998565,18.661111
3,GB,Cincinnati Bengals Defense,TE,7.305887,14.094113,16.023692,16.963096,17.902501
4,GB,Cincinnati Bengals Defense,K,-0.393503,2.293503,0.838455,1.195958,1.553460


In [7]:
df

,Unnamed: 0,Opponent,Points_Against,Position,Team,Week,Year,Season_Ave,Season_STD,Home_Away,Game_Dev
0,0,IND,35.50,QB,Detroit Lions Defense,1,2016,18.97750,9.830612,Away,0.870636
1,0,NO,31.42,QB,Oakland Raiders Defense,1,2016,17.40500,8.353195,Away,0.805228
2,0,KC,28.02,QB,Los Angeles Chargers Defense,1,2016,16.15500,7.159565,Away,0.734448
3,0,DET,26.10,QB,Indianapolis Colts Defense,1,2016,17.69000,8.643188,Home,0.475410
4,0,TB,25.54,QB,Atlanta Falcons Defense,1,2016,19.08125,9.153823,Home,0.338487
5,0,ATL,24.36,QB,Tampa Bay Buccaneers Defense,1,2016,16.30250,9.332471,Away,0.494249
6,0,GB,23.56,QB,Jacksonville Jaguars Defense,1,2016,15.21125,6.357343,Home,0.548854
7,0,OAK,22.36,QB,New Orleans Saints Defense,1,2016,18.26875,7.266412,Home,0.223948
8,0,PIT,21.80,QB,Washington Redskins Defense,1,2016,16.60125,7.000641,Home,0.313154
9,0,CAR,21.16,QB,Denver Broncos Defense,1,2016,12.12000,6.023230,Home,0.745875
